# Le CLOUSURE

Sono funzione interne che accedono allo scopo del loro contentiore

In [13]:
def make_adder(n):
    def adder(x):
        return x + n
    return adder

In [14]:
uno = make_adder(5)
uno(10)

15

La funzione adder accede ad n (sola lettura), paramentro della funzione contenitrice make_adder. Lo statement nolocal può essere utilizzato per rendere la variabile accessile anche in scrittura.

## Esistono 3 tipologie di decoratori

- decoratori che eseguono la funzione solo quando viene definita
- decoratori che sostituiscono ad una funzione un'altra funzione
- decoratori che **incartano** un'altra funzione aggiungendo elementi prima e dopo

##  1 - Decoratori del primo tipo

In [15]:
def base1():
    print("io sono la funzione originale")

In [16]:
def deco1(func):
    print("faccio cose")
    return func

Questo decoratore viene applicato SOLO 1 volta, quando generiamo la funzione dopo aver importato il modulo!

In [17]:
base1()

io sono la funzione originale


In [18]:
@deco1
def base1():
    print("io sono la funzione originale")

faccio cose


In [19]:
__all__ = []
def public(func):
    '''Add the func to __all__'''
    __all__.append(func)
    return func

def funzione_uno():
    pass

@public
def funzione_due():
    pass
@public
def funzione_tre():
    pass

In questo caso dopo un'importazione `from file import * ` solo le funzioni decorate con public saranno importate. 
Ricordiamo che l'importazione di un modulo con * prenderà solo i moduli presenti in __all__ e li inserirà nel nostro file py.

In [20]:
def private(func):
    func.private = True
    return func

@private
def secret():
    return 'segreta'


def not_secret():
    return 'non segreta'


for func in secret, not_secret:
    is_private = getattr(func, 'private', False)
    if not is_private:
        print(func())

non segreta


## 2-  Decoratori secondo tipo: Sostituzione funzione

In [21]:
def deco(func):
    def funzione():
        print("funzione sostitutiva")
    return funzione

In [22]:

def base2():
    print("io sono la funzione originale")

In [24]:
base2()

io sono la funzione originale


In [25]:
@deco
def base2():
    print("io sono la funzione originale")

In [26]:
base2()

funzione sostitutiva


La funzione base viene sostituita, essendo decorata, dalla funzione sostitutiva

In [27]:
import getpass

def check_root(func):
    if getpass.getuser() != 'root':
        def warn_user(*args, **kwargs):
            msg = 'Devi essere utente root per eseguire {}'.format(func.__name__)
            raise UserWarning(msg)
        return warn_user
    else:
        return func


@check_root
def comando():
    print("Eseguo il comando solo se sono root")


In questo caso se utente root esegue la funzione, diversamente la sostituisce con una funzione che mostra un messaggio di avvertimento

## 3- Decoratori terzo tipo: Wrapper

In [28]:
def deco(func):
    def wrapper(*args, **kwargs):
        print("faccio cose prima")
        func(*args, **kwargs)
        print("faccio cose dopo")
    return wrapper

def stampa():
    print("stampata")

stampa()

stampata


L'indicazione di args e kwargs permette il funzionamento del decoratore con TUTTE le funzioni, sia quelle che hanno parametri che quelle senza parametri.

In [29]:
@deco
def stampa():
    print("stampata")

In [30]:
stampa()

faccio cose prima
stampata
faccio cose dopo


In [31]:
import datetime

def log_calls(func):
    fname = func.__name__
    def logger(*args):
        t = datetime.datetime.now().isoformat(' ')
        res = func(*args)
        a = ', '.join(map(str, args))
        print('[{}] {} ({}) -> {}'.format(t, fname, a,  res))
        return res
    return logger

@log_calls
def somma(a, b):
    return a + b


print(somma(1,5))


[2022-05-05 18:14:22.790162] somma (1, 5) -> 6
6


In [32]:
import time

def calc_time(func):
    def inner(*args, **kwargs):
        start = time.time()
        res = func(*args, **kwargs)
        end = time.time()
        print(
f'''
La funzione ha impiegato 
    {end - start:.5f} secondi
per essere eseguita''')
        return res
    return inner

@calc_time
def produce_lista(n):
    import sys
    lista = [x for x in range(n)]
    return f'''
    {sys.getsizeof(lista):,} <- lista in byte'''


In [33]:
print(produce_lista(100))


La funzione ha impiegato 
    0.00004 secondi
per essere eseguita

    904 <- lista in byte


In [34]:
def memoize(func):
    cache = {}
    def inner(*args, **kwargs):
        if args in cache:
            print("ritorno risultati in cache")
            return cache[args]
        res = func(*args, **kwargs)
        cache[args] = res
    return inner

@memoize
def crea_lista(n):
    lista = [x for x in range(n)]
    print("lista completata")
    return lista

# Decorator Factory

Sono funzioni che ritornano dei DECORATORI

In [35]:
def deco_factory(extra_args):
    def deco(func):
        def inner(*args, **kwargs):
            pass
        return inner
    return deco

- deco_factory = funzione che genera il decoratore
- deco = il decoratore
- inner = funzione che sarà decorata

Utilizzo: fornire alla funzione inner degli argomenti extra, presi dallo scope più esterno, quello del decorator factory

In [36]:
def repeat_for(times):
    def deco(func):
        def inner(*args):
            for x in range(times):
                func()
        return inner
    return deco


@repeat_for(5)
def alpha():
    print("ciao amico mio")

In [37]:
def moltiplicatore(times):
    def deco(func):
        def inner(*args):
            for x in range(times):
                func(), print("Andrea")
        return inner
    return deco

@moltiplicatore(5)
def stampa():
    print("ciao amico mio...",end=" ")





stampa()

ciao amico mio... Andrea
ciao amico mio... Andrea
ciao amico mio... Andrea
ciao amico mio... Andrea
ciao amico mio... Andrea


Quando applichiamo un decoratore PERDIAMO tutte le informazioni della funzione originale:

- nome
- docstring
- help

Utilizzando le @functools.wraps possiamo mantenere tutte queste informazioni

In [38]:
from functools import wraps

def deco(func):
    @wraps(func)
    def inner(*args, **kwargs):
        pass
    return inner


In [39]:
from functools import wraps

def log_calls(func):
    fname = func.__name__
    @wraps(func)
    def logger(*args, **kwargs):
        print(fname, 'chiamata')
        return func(*args, **kwargs)
    return logger

@log_calls
def add(x, y):
    """Return the sum of x and y"""
    return x + y


print(add(1,1))
print("docstring:", add.__doc__)
print("nome funzione:", add.__name__)
print("modulo funzione:", add.__module__)
print("funzione wrapped:", add.__wrapped__)

add chiamata
2
docstring: Return the sum of x and y
nome funzione: add
modulo funzione: __main__
funzione wrapped: <function add at 0x7f8ab41b4160>


Queste informazioni, se avessimo decorato senza uso di wraps, si sarebbero perse

> USARE wraps permette di non tener conto dell'ordine con cui sono stati applicati i decoratori!

# Combinare i decoratori

Alle funzioni possono essere associati più decoratori contemporaneamente; l'ordine andrà da sotto a sopra!

In [40]:
def decoratore1(func):
    print("applico decoratore 1")
    return func

In [41]:
def decoratore2(func):
    print("applico decoratore 2")
    return func

In [42]:
def decoratore3(func):
    print("applico decoratore 3")
    return func

In [43]:
@decoratore1
@decoratore2
@decoratore3
def somma(x,y):
    return x + y

somma(1,1)

applico decoratore 3
applico decoratore 2
applico decoratore 1


2

## In sintesi da ricordare per quanto riguarda i *Decoratori*

- They are applied in reverse order
    - the last decorator is applied first
    - every decorator is applied to the function returned by the next deco
- for some decorators the order matters, for others it doesn't
- @wraps can improve decorators interaction